# Semantic Segmentation - Automatic Data Augmentation testing and Analysis

## Setup - Colab

In [ ]:
from google.colab import drive
from google.colab import files as colab_files
drive.mount("/content/gdrive")

## Setup - Python

In [ ]:
import cv2
import glob
import os
import numpy as np
import random
import math
import time
import shutil
from matplotlib.pyplot import imshow
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

In [ ]:
SIZE = 256
STRIDE = 64
REMOVE_NOTHING_CHANCE = 75 # chance of removing a clear image

In [ ]:
WORK_DIR = "./project"

## Setup - Dependencies

In [ ]:
!git clone https://github.com/divamgupta/image-segmentation-keras
%cd image-segmentation-keras
!python setup.py install

In [ ]:
import keras_segmentation

## Setup - Functions

### Dataset related

In [ ]:
# creates a new dataset folder and a dataset-gen.zip file
def develop_dataset():
    reset_dataset()
    os.system("unzip \"%s/dataset.zip\"" % (WORK_DIR))
    os.system("mkdir \
            dataset/train-x dataset/train-y \
            dataset/train-processed \
            out \
            tmp")
    convert_png()
    cut_images(STRIDE) # clear_dataset() is already called in here
    verify_data()
    os.system("zip -0 -r dataset-gen.zip dataset/ && \
                mv dataset-gen.zip \"%s/\"" % (WORK_DIR))

# unzips the generated dataset folder from the dataset-gen zip file
def create_dataset():
    os.system("unzip \"%s/dataset-gen.zip\"" % (WORK_DIR))
    verify_data()

# resets the dataset folder
def reset_dataset():
    os.system("rm -rf dataset")
    create_dataset()

def verify_data():
    train_x = 0 # train input files quantity
    train_p = 0 # train processed files quantity
    for file in glob.iglob("dataset/train-x/*.png"):
        train_x += 1
    for file in glob.iglob("dataset/train-processed/*.png"):
        train_p += 1

    print("[Verify data] train_x", train_x, ", train_p", train_p)
    if train_x != train_p:
        print("[ERROR] Dataset input and output quantity isn't synced")

# move the output folder
def move_out(folder):
    os.rename("out", "%s/%s" % (WORK_DIR, folder))

# converts all the dataset images from .tif to .png
def convert_png():
    for file in glob.iglob("dataset/*/**.tif"):
        img = cv2.imread(file, 0)
        cv2.imwrite(file.replace(".tif", ".png"), img)

# clears the dataset (removes some images)
def clear_dataset():
    for file in glob.iglob("dataset/train-y/*.png"):
        img = cv2.imread(file, 0)
        if np.sum(img) == 0:
            if random.randint(0,100) <= REMOVE_NOTHING_CHANCE:
                filename = file.split("/")[-1]
                file_x = "dataset/train-x/" + filename
                file_y = file

                os.system("rm \"%s\" \"%s\"" % (file_x, file_y))

# resets the out folder
def reset_out():
    if os.path.exists("out"):
        shutil.rmtree("out")
    if os.path.exists("tmp"):
        shutil.rmtree("tmp")
    os.mkdir("tmp")
    os.mkdir("out")
    os.mkdir("out/src")
    os.mkdir("out/split")
    os.mkdir("out/over")

In [ ]:
def extend_image(file):
    img = cv2.imread(file, 0)
    height = len(img)
    width = len(img[0])

    new_height = math.ceil(height/SIZE) * SIZE
    new_width = math.ceil(width/SIZE) * SIZE

    img_out = np.zeros((new_height,new_width,1))
    
    # copies the image
    for y in range(height):
        for x in range(width):
            img_out[y,x] = img[y,x]
    
    # expands the right area
    for y in range(0, height):
        for x in range(width, new_width):
            img_out[y,x] = img[y,width-1]
    
    # expands the bottom area
    for y in range(height, new_height):
        for x in range(0, width):
            img_out[y,x] = img[height-1,x]
    
    # expands the bottom-right area
    for y in range(height, new_height):
        for x in range(width, new_width):
            img_out[y,x] = img[height-1,width-1]

    cv2.imwrite(file, img_out)

In [ ]:
# it's bugged, may or may not work
def _download(folder, download_as=None):
    if download_as is None:
        download_as = folder
    if "/" in download_as:
        download_as = download_as.split("/")[-1]
    
    zipped = download_as + ".zip"
    
    if os.path.exists(download_as):
        shutil.rmtree(download_as)
    shutil.copytree(folder, download_as)
    os.system("zip -0 -r \"%s\" \"%s\"" % (zipped, download_as))

    #colab_files.download(zipped)
    os.rename(zipped, "%s/%s" % (WORK_DIR, zipped))
    os.remove(zipped)
    shutil.rmtree(download_as)

In [ ]:
def download(folder, download_as=None):
    if download_as is None:
        download_as = folder
    if "/" in download_as:
        download_as = download_as.split("/")[-1]

    if not os.path.exists("%s/resultados" % (WORK_DIR)):
        os.mkdir("%s/resultados" % (WORK_DIR))
        
    dest = "%s/resultados/%s" % (WORK_DIR, download_as)
    if os.path.exists(download_as):
        shutil.rmtree(download_as)
    if os.path.exists(dest):
        shutil.rmtree(dest)
    
    shutil.copytree(folder, dest)

### Image utils

In [ ]:
# cut a single image and output its parts to 'outfolder' 
def cut_image(file, outfolder, stride=STRIDE):
    filename = file.split("/")[-1]
    index = 0
    img = cv2.imread(file, 0)
    height = len(img)
    width = len(img[0])
    for y in range(0, height, stride):
        for x in range(0, width, stride):
            img_out = np.zeros((SIZE,SIZE,1))
            for i in range(SIZE): # y
                for j in range(SIZE): # x
                    if i+y < height and j+x < width:
                        img_out[i,j] = img[i+y,j+x]
            cv2.imwrite("%s/%06d_%s" % (outfolder, index, filename), img_out)
            index += 1

# returns a noised image
def noise(image, var):
    row,col,ch = image.shape
    mean = 0
    gauss = np.random.normal(mean,var,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss

    cv2.normalize(noisy, noisy, 0, 255, cv2.NORM_MINMAX, dtype=-1)
    noisy = noisy.astype(np.uint8)
    return noisy

### Image core

In [ ]:
def cut_images(stride):
    for file_x in glob.iglob("dataset/input/*.png"):
        filename = file_x.split("/")[-1]
        file_y = "dataset/label/%s" % (filename)

        extend_image(file_x)
        extend_image(file_y)

        cut_image(file_x, "dataset/train-x")
        cut_image(file_y, "dataset/train-y")
    
    for file_x in glob.iglob("dataset/test-input/*.png"):
        filename = file_x.split("/")[-1]
        file_y = "dataset/test-label/%s" % (filename)

        extend_image(file_x)
        extend_image(file_y)
    
    clear_dataset()
    process_label()

def process_label():
    for file in glob.iglob("dataset/train-y/*.png"):
        filename = file.split("/")[-1]
        img = cv2.imread(file, 0)
        for i in range(len(img)):
            for j in range(len(img[i])):
                if(img[i,j] > 0): img[i,j] = 1
                else: img[i,j] = 0

        cv2.imwrite("dataset/train-processed/"+filename, img)

# returns float iou score for a single test file
def get_iou(filename):
    file_out = "out/src/out_%s" % (filename)
    file_target = "dataset/test-label/%s" % (filename)

    if not os.path.isfile(file_out):
        print("[Error/IOU Score] Out file not found")
        return -1
    if not os.path.isfile(file_target):
        print("[Error/IOU Score] Target file not found")
        return -1

    out = cv2.imread(file_out, 0)
    target = cv2.imread(file_target, 0)

    # Evaluates the intersection "area"
    intersect = 0
    for i in range(len(target)):
        for j in range(len(target[i])):
            if(out[i,j] == 255 and target[i,j] == 255):
                intersect += 1
    
    # Evaluates the union "area"
    union = 0
    for i in range(len(target)):
        for j in range(len(target[i])):
            if(out[i,j] == 255 or target[i,j] == 255):
                union += 1

    if union == 0:
        union += 1
        intersect += 1

    iou_score = intersect/union
    return iou_score

### Data Augmentation core

In [ ]:
def invert_images(folder):
    for file in glob.iglob("%s/*" % (folder)):
        img = cv2.imread(file, 0)
        for i in range(len(img)):
            for j in range(len(img[i])):
                img[i,j] = abs(255-img[i,j])
        cv2.imwrite(file, img)

def normalize_images(folder):
    for file in glob.iglob("%s/*" % (folder)):
        img = cv2.imread(file)
        cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX, dtype=-1)
        cv2.imwrite(file, img)

def noise_images(folder, chance, var=5):
    for file in glob.iglob("%s/*" % (folder)):
        if random.randint(0,100) > chance: continue
        img = cv2.imread(file)
        img = noise(img, var)
        cv2.imwrite(file, img)

def flip_images(chance):
    for file_x in glob.iglob("dataset/train-x/*.png"):
        if random.randint(0,100) > chance: continue
        file_y = file_x.replace("train-x", "train-processed")
        
        img = cv2.imread(file_x)
        img = cv2.flip(img, 0)
        cv2.imwrite(file_x.replace(".png", "fp.png"), img)

        img = cv2.imread(file_y)
        img = cv2.flip(img, 0)
        cv2.imwrite(file_y.replace(".png", "fp.png"), img)
        

def rotate_images(chance):
    for file_x in glob.iglob("dataset/train-x/*.png"):
        if random.randint(0,100) > chance: continue
        file_y = file_x.replace("train-x", "train-processed")
        
        img = cv2.imread(file_x)
        img = np.rot90(img)
        img = np.rot90(img)
        cv2.imwrite(file_x.replace(".png", "fp.png"), img)

        img = cv2.imread(file_y)
        img = np.rot90(img)
        img = np.rot90(img)
        cv2.imwrite(file_y.replace(".png", "fp.png"), img)

## Setup - Model

### Model functions

In [ ]:
def test_model():
    reset_out()
    
    for file in glob.iglob("dataset/test-input/*.png"):
        cut_image(file, "tmp", stride=SIZE)
        filename = file.split("/")[-1]
        img_original = cv2.imread(file, 0)
        break_width = math.ceil(len(img_original[0])/SIZE) # ceil because there's a padding at the left of the cropped image

        img_in = None
        img_in_x = None
        img_out = None
        img_out_x = None
        current_width = 0
        for file_in in sorted(glob.iglob("tmp/*")):
            model.predict_segmentation(
                inp=file_in,
                out_fname="tmp_out.png"
            )
            temp_in = cv2.imread(file_in, 0)
            temp_out = cv2.imread("tmp_out.png", 0)

            if img_out_x is None:
                img_out_x = temp_out
                img_in_x = temp_in
            else:
                img_out_x = np.append(img_out_x, temp_out, axis=1)
                img_in_x = np.append(img_in_x, temp_in, axis=1)

            current_width += 1

            if current_width == break_width:
                if img_out is None:
                    img_out = img_out_x
                    img_in = img_in_x
                else:
                    img_out = np.append(img_out, img_out_x, axis=0)
                    img_in = np.append(img_in, img_in_x, axis=0)
                img_out_x = None
                img_in_x = None
                current_width = 0
        
        for i in range(len(img_out)):
            for j in range(len(img_out[i])):
                if(img_out[i,j] > 200): img_out[i,j] = 255
                else: img_out[i,j] = 0
            
        os.system("rm tmp/*")

        cv2.imwrite("out/src/out_%s" % (filename), img_out)
        cv2.imwrite("out/src/in_%s" % (filename), img_in)

        process_out(filename)

In [ ]:
def process_out(file):
    img_in = cv2.imread("out/src/in_%s" % (file), 1)
    img_out = cv2.imread("out/src/out_%s" % (file), 1)
    img_target = cv2.imread("dataset/test-label/%s" % (file), 1)
    
    # Gets the iou score
    iou_score = get_iou(file)
    if iou_score == -1:
        print("Error while checking iou_score")
        return
    
    # Creates/appends the description file
    create_header = False
    if not os.path.isfile("out/description.csv"):
        create_header = True
    with open("out/description.csv", "a") as desc:
        if create_header:
            desc.write("name,iou\n")
        desc.write("%s,%f\n" % (file, iou_score))

    # Creates the "splitscreen" image
    img = np.append(img_in, img_out, axis=1)
    cv2.imwrite("out/split/%s" % (file), img)

    # Creates the overlayed image (in and out)
    for i in range(len(img_out)):
        for j in range(len(img_out[i])):
            if img_out[i,j][0] == 255:
                img_out[i,j] = [0, 0, 255] # B,G,R
    img = cv2.addWeighted(img_in,1.0,img_out,0.2,0)
    cv2.imwrite("out/%s" % (file), img)

    # Creates the overlayed image (out and target)
    img = cv2.addWeighted(img_target,0.85,img_out,0.7,0)
    cv2.imwrite("out/over/%s" % (file), img)

In [ ]:
model = keras_segmentation.models.segnet.segnet(n_classes=2, input_height=SIZE, input_width=SIZE)

In [ ]:
def reset_all():
    model = keras_segmentation.models.segnet.segnet(n_classes=2, input_height=SIZE, input_width=SIZE)
    reset_dataset()

In [ ]:
def log_out(texts):
    with open("out/description.txt", "w") as file:
        file.write("\n".join(texts))

In [ ]:
def train():
    model.train( 
        train_images =  "dataset/train-x/",
        train_annotations = "dataset/train-processed/",
        checkpoints_path = "%s/model/model-ckpt.h5" % (WORK_DIR) , epochs=50
    )

In [ ]:
def fetch_model_results():
    results_folder = "%s/resultados" % (WORK_DIR)
    model_file = "%s/model.csv" % (WORK_DIR)

    if not os.path.isfile(model_file):
        with open(model_file, "w") as file:
            file.write("name,iou,augmentations\n")

    def get_file_lines(file_name):
        lines = []
        with open(file_name, "r") as file:
            lines = file.readlines()
        for i in range(len(lines)):
            lines[i] = lines[i].replace("\n","")
        return lines

    for root, dirs, files in os.walk(results_folder, topdown = False):
        for name in files:
            if name == "description.csv":
                path = os.path.join(root, name)
                
                csv_file = os.path.join(root, name)
                txt_file = os.path.join(root, "description.txt")

                # get the augmentations
                txt_lines = get_file_lines(txt_file)
                augmentations = "/".join(txt_lines)

                # get the csv lines from training and append them with the augmentations
                csv_lines = get_file_lines(csv_file)
                csv_lines.pop(0)
                for i in range(len(csv_lines)):
                    csv_lines[i] = csv_lines[i] + "," + augmentations
                
                # write the results to the main model csv file
                with open(model_file, "a") as file:
                    for l in csv_lines:
                        file.write(l + "\n")
                

## Functions

### Base

In [ ]:
create_dataset()

### Processes

In [ ]:
reset_dataset()
train()
test_model()
log_out(["limpo"])
download("out", "limpo")

In [ ]:
reset_all()
normalize_images("dataset/train-x")
normalize_images("dataset/test-input")
train()
test_model()
log_out(["normalizado"])
download("out", "normalizado")

In [ ]:
reset_all()
normalize_images("dataset/test-input")
train()
test_model()
log_out(["teste_normalizado"])
download("out", "teste_normalizado")

In [ ]:
reset_all()
noise_images("dataset/train-x", 50, var=5)
train()
test_model()
log_out(["ruido 50% σ²=5"])
download("out", "ruido")

In [ ]:
reset_all()
normalize_images("dataset/train-x")
normalize_images("dataset/test-input")
noise_images("dataset/train-x", 50, var=5)
train()
test_model()
log_out(["normalizado", "ruido 50% σ²=5"])
download("out", "normalizado ruido")

In [ ]:
reset_all()
normalize_images("dataset/train-x")
normalize_images("dataset/test-input")
noise_images("dataset/train-x", 50, var=12)
train()
test_model()
log_out(["normalizado", "ruido 50% σ²=12"])
download("out", "normalizado ruido12")

In [ ]:
reset_all()
normalize_images("dataset/train-x")
normalize_images("dataset/test-input")
flip_images(50)
train()
test_model()
log_out(["normalizado", "espelhado 50%"])
download("out", "normalizado espelhado50")

In [ ]:
reset_all()
normalize_images("dataset/train-x")
normalize_images("dataset/test-input")
flip_images(100)
train()
test_model()
log_out(["normalizado", "espelhado 100%"])
download("out", "norm espelhado100")

In [ ]:
reset_all()
normalize_images("dataset/train-x")
normalize_images("dataset/test-input")
flip_images(50)
invert_images("dataset/train-x")
invert_images("dataset/test-input")
train()
test_model()
log_out(["normalizado", "espelhado 50%", "invertido"])
download("out", "normalizado espelhado invertido")

In [ ]:
reset_all()
normalize_images("dataset/train-x")
normalize_images("dataset/test-input")
flip_images(50)
noise_images("dataset/train-x", 50, var=5)
train()
test_model()
log_out(["normalizado", "espelhado 50%", "ruido σ²=5"])
download("out", "normalizado espelhado ruido5")

In [ ]:
reset_all()
normalize_images("dataset/train-x")
normalize_images("dataset/test-input")
flip_images(50)
noise_images("dataset/train-x", 50, var=3)
train()
test_model()
log_out(["normalizado", "espelhado 50%", "ruido σ²=3"])
download("out", "normalizado espelhado ruido3")

In [ ]:
reset_all()
rotate_images(50)
flip_images(20)
noise_images("dataset/train-x", 50, var=5)
normalize_images("dataset/train-x")
normalize_images("dataset/test-input")
train()
test_model()
log_out(["tudo"])
download("out", "tudo")

In [ ]:
fetch_model_results()

# Data Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
models_csv_paths = ["model1.csv"]
titles = ["SegNet"]

In [ ]:
df = pd.read_csv(models_csv_paths[0])
augs = []
for x in df['augmentations']:
    if x not in augs:
        augs.append(x)

In [ ]:
fig, ax = plt.subplots(nrows=len(augs), ncols=len(models_csv_paths), sharey='all', figsize=(3.5*len(models_csv_paths), 2.5*(len(augs))))
plt.tight_layout()

In [ ]:
for modelid, csv_file in enumerate(models_csv_paths):
    df = pd.read_csv(csv_file)
    df = df.sort_values('name')
    
    for i, a in enumerate(augs):
        iou_scores = df.loc[df['augmentations'] == a]['iou']
        iou_scores = iou_scores.values

        title = ""
        x = np.arange(1,len(iou_scores)+1)
        y = iou_scores

        for j in range(len(iou_scores)):
            if y[j] == 0:
                if len(models_csv_paths) == 1: ax[i].scatter(x[j], y[j], color='red')
                else: ax[i, modelid].scatter(x[j], y[j], color='red')
            else:
                if len(models_csv_paths) == 1: ax[i].scatter(x[j], y[j], color='blue')
                else: ax[i, modelid].scatter(x[j], y[j], color='blue')
        if i == 0:
            for t in titles[modelid].split(" "):
                title += r"$\bf{" + t + "}$ "
            title = title + "\n\n" + a.replace("_", " ")
            
        else:
            title = a.replace("_", " ")
        
        if len(models_csv_paths) == 1:
            ax[i].set_title(title)

            ax[i].set_xlabel("Número da imagem")
            ax[i].set_ylabel("iou score")
        else:
            ax[i, modelid].set_title(title)

            ax[i, modelid].set_xlabel("Número da imagem")
            ax[i, modelid].set_ylabel("iou score")

In [ ]:
fig.tight_layout()
fig.subplots_adjust(hspace=0.7)
fig.savefig("%s/out.png" % (WORK_DIR))

###